In [4]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\PC\Data\ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [5]:
save_dir=f"{exps_dir}/exp_prj_vehicle_loan_prediction/exp_nosmote"

os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [6]:
x_train=pd.read_csv(f'{save_dir}/x_train.csv')
y_train=pd.read_csv(f'{save_dir}/y_train.csv')
x_test=pd.read_csv(f'{save_dir}/x_test.csv')
y_test=pd.read_csv(f'{save_dir}/y_test.csv')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(244730, 45) (244730, 1) (76941, 45) (76941, 1)


### (2) Lựa chọn các đặc trưng

In [7]:
x_train

,UNIQUEID,DISBURSED_AMOUNT,ASSET_COST,LTV,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,DISBURSED_AMOUNT_new,DISBURSED_AMOUNT_bins,ASSET_COST_new,ASSET_COST_bins,LTV_new,LTV_bins,PERFORM_CNS_SCORE_bins,PRI_NO_OF_ACCTS_new,PRI_NO_OF_ACCTS_bins,PRI_OVERDUE_ACCTS_new,PRI_OVERDUE_ACCTS_bins,age,disbursal_time
0,0.845535,-0.042843,-0.151742,0.055274,2,0,1,0,0,0,0,-0.854883,13,-0.461680,0,0,-0.176955,-0.079264,-0.079019,0,0,0,-0.033943,-0.045345,-0.04472,-0.084976,-0.022927,0,0,-0.589057,-0.568454,0,0.005415,3,0.513786,1,0,3,3,0,1,0,1,1.839794,-1.007492
1,-1.191835,0.028015,-0.680741,1.253429,2,0,1,0,0,0,0,-0.854883,13,-0.461680,0,0,-0.176955,-0.079264,-0.079019,0,0,0,-0.033943,-0.045345,-0.04472,-0.084976,-0.022927,0,0,-0.589057,-0.568454,1,0.088563,1,-0.540575,2,0,0,3,0,1,0,1,2.069620,1.188236
2,-0.082136,-0.626317,-0.430638,-0.328694,2,0,0,1,1,0,0,-0.854883,13,-0.461680,0,0,-0.176955,-0.079264,-0.079019,0,0,0,-0.033943,-0.045345,-0.04472,-0.084976,-0.022927,0,0,-0.589057,-0.568454,0,-0.679264,2,-0.042089,3,0,3,3,0,1,0,1,-0.480244,0.017181
3,1.042062,0.795720,0.890423,-0.135837,2,0,1,0,0,0,0,1.127075,5,-0.083744,1,1,-0.176106,-0.072046,-0.078636,0,0,0,-0.033943,-0.045345,-0.04472,-0.062714,-0.022927,1,1,0.666672,0.693513,0,0.989427,0,2.590948,0,0,3,0,0,0,0,0,1.650181,-1.080682
4,1.355240,-2.128507,-0.231695,-3.236380,2,0,1,0,0,0,0,1.457894,1,-0.272712,1,0,1.391249,0.471082,0.471187,0,0,0,-0.033943,-0.045345,-0.04472,-0.084976,-0.022927,0,0,2.120675,0.868786,0,-2.442010,2,0.354429,1,0,2,0,0,1,0,1,0.886981,-1.227064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244725,-0.267450,0.180154,-0.389518,0.894768,1,0,1,0,0,0,0,1.207623,3,-0.020232,1,0,-0.168332,-0.074029,-0.073785,0,0,0,-0.033943,-0.045345,-0.04472,-0.037092,-0.022927,1,0,-0.191963,-0.158999,1,0.267091,1,0.039868,3,0,0,0,0,0,0,1,0.126925,0.222634
244726,1.430012,0.161941,0.096449,-0.006764,1,0,1,0,0,0,0,0.983245,7,-0.067345,0,0,-0.176955,-0.079264,-0.079019,0,0,0,-0.033943,-0.045345,-0.04472,-0.084976,-0.022927,0,0,-0.120685,-0.106660,0,0.245718,1,1.008460,1,0,3,2,0,0,0,1,0.409218,-1.266835
244727,1.496122,0.276513,0.260907,-0.049895,0,0,1,0,0,0,0,-0.854883,13,-0.461680,0,0,-0.176955,-0.079264,-0.079019,0,0,0,-0.033943,-0.045345,-0.04472,-0.084976,-0.022927,0,0,-0.589057,-0.568454,0,0.380164,1,1.336244,0,0,3,3,0,1,0,1,-1.490077,-1.300932
244728,0.502600,0.235246,-0.154216,0.591123,2,0,0,1,1,0,0,-0.854883,13,-0.461680,0,0,-0.176955,-0.079264,-0.079019,0,0,0,-0.033943,-0.045345,-0.04472,-0.084976,-0.022927,0,0,-0.589057,-0.568454,0,0.331739,1,0.508854,1,0,1,3,0,1,0,1,-0.496636,-0.663539


In [8]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [9]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo                   Feature  Coefficient
5       MOBILENO_AVL_FLAG          0.0
36                LTV_new          0.0
39    PRI_NO_OF_ACCTS_new          0.0
41  PRI_OVERDUE_ACCTS_new          0.0
Xếp hạng các đặc trưng theo mức độ quan trọng:
                                Feature  Coefficient
9                          DRIVING_FLAG    -2.975837
6                           AADHAR_FLAG    -2.250412
8                          VOTERID_FLAG    -1.937265
0                              UNIQUEID     1.065718
44                       disbursal_time     0.994094
42               PRI_OVERDUE_ACCTS_bins    -0.790253
10                        PASSPORT_FLAG    -0.785492
7                              PAN_FLAG    -0.698413
3                                   LTV     0.509708
1                      DISBURSED_AMOUNT    -0.362058
40                 PRI_NO_OF_ACCTS_bins    -0.358642
2                            ASSET_COST     0.309482
30                CREDIT_HISTORY_LENGTH    -0.285840
43

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [10]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [11]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['AADHAR_FLAG', 'DRIVING_FLAG', 'PAN_FLAG', 'PASSPORT_FLAG', 'PRI_DISBURSED_AMOUNT', 'PRI_OVERDUE_ACCTS_bins', 'PRI_SANCTIONED_AMOUNT', 'UNIQUEID', 'VOTERID_FLAG', 'disbursal_time']
RFE
Kiểm tra mô hình LDA
['AADHAR_FLAG', 'DRIVING_FLAG', 'PAN_FLAG', 'PASSPORT_FLAG', 'PRI_DISBURSED_AMOUNT', 'PRI_OVERDUE_ACCTS_bins', 'PRI_SANCTIONED_AMOUNT', 'UNIQUEID', 'VOTERID_FLAG', 'disbursal_time']
RFE
Kiểm tra mô hình DTC
['ASSET_COST', 'ASSET_COST_new', 'CREDIT_HISTORY_LENGTH', 'DISBURSED_AMOUNT', 'DISBURSED_AMOUNT_new', 'LTV', 'PRI_NO_OF_ACCTS', 'UNIQUEID', 'age', 'disbursal_time']
RFE
Kiểm tra mô hình RD
['ASSET_COST', 'ASSET_COST_new', 'DISBURSED_AMOUNT', 'DISBURSED_AMOUNT_new', 'LTV', 'PERFORM_CNS_SCORE', 'PRI_NO_OF_ACCTS', 'UNIQUEID', 'age', 'disbursal_time']
RFE
Kiểm tra mô hình LGBM
[LightGBM] [Info] Number of positive: 122365, number of negative: 122365
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020825 seconds.
You can s

### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [12]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'DISBURSED_AMOUNT_new', 'DISBURSED_AMOUNT_bins', 'LTV_bins', 'PERFORM_CNS_SCORE_bins', 'PRI_OVERDUE_ACCTS_bins'], dtype='object')
Lua chon dac trung GNB
Index(['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'DISBURSED_AMOUNT_new', 'DISBURSED_AMOUNT_bins', 'LTV_bins', 'PERFORM_CNS_SCORE_bins', 'PRI_OVERDUE_ACCTS_bins'], dtype='object')
Lua chon dac trung NN
Index(['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'DISBURSED_AMOUNT_new', 'DISBURSED_AMOUNT_bins', 'LTV_bins', 'PERFORM_CNS_SCORE_bins', 'PRI_OVERDUE_ACCTS_bins'], dtype='object')
Lua chon dac trung BG
Index(['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'DISBURSED_AMOUNT_new', 'DIS

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [13]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'DISBURSED_AMOUNT_new', 'DISBURSED_AMOUNT_bins', 'LTV_bins', 'PERFORM_CNS_SCORE_bins', 'PRI_OVERDUE_ACCTS_bins'], 'GNB': ['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'DISBURSED_AMOUNT_new', 'DISBURSED_AMOUNT_bins', 'LTV_bins', 'PERFORM_CNS_SCORE_bins', 'PRI_OVERDUE_ACCTS_bins'], 'NN': ['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'DISBURSED_AMOUNT_new', 'DISBURSED_AMOUNT_bins', 'LTV_bins', 'PERFORM_CNS_SCORE_bins', 'PRI_OVERDUE_ACCTS_bins'], 'BG': ['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'DISBURSED_AMOUNT_new', 'DISBURSED_AMOUNT_bins', 'LTV_bins', 'PERFORM_CNS_SCORE_bins', 'PRI_OVERDUE_ACCTS_bins'], 'NB': ['DISBURSED_AMOUNT', 'LTV', 'AADHAR_FLAG', 

In [14]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['AADHAR_FLAG',
  'DRIVING_FLAG',
  'PAN_FLAG',
  'PASSPORT_FLAG',
  'PRI_DISBURSED_AMOUNT',
  'PRI_OVERDUE_ACCTS_bins',
  'PRI_SANCTIONED_AMOUNT',
  'UNIQUEID',
  'VOTERID_FLAG',
  'disbursal_time'],
 'LDA': ['AADHAR_FLAG',
  'DRIVING_FLAG',
  'PAN_FLAG',
  'PASSPORT_FLAG',
  'PRI_DISBURSED_AMOUNT',
  'PRI_OVERDUE_ACCTS_bins',
  'PRI_SANCTIONED_AMOUNT',
  'UNIQUEID',
  'VOTERID_FLAG',
  'disbursal_time'],
 'DTC': ['ASSET_COST',
  'ASSET_COST_new',
  'CREDIT_HISTORY_LENGTH',
  'DISBURSED_AMOUNT',
  'DISBURSED_AMOUNT_new',
  'LTV',
  'PRI_NO_OF_ACCTS',
  'UNIQUEID',
  'age',
  'disbursal_time'],
 'RD': ['ASSET_COST',
  'ASSET_COST_new',
  'DISBURSED_AMOUNT',
  'DISBURSED_AMOUNT_new',
  'LTV',
  'PERFORM_CNS_SCORE',
  'PRI_NO_OF_ACCTS',
  'UNIQUEID',
  'age',
  'disbursal_time'],
 'LGBM': ['AVERAGE_ACCT_AGE',
  'CREDIT_HISTORY_LENGTH',
  'DISBURSED_AMOUNT',
  'LTV',
  'PERFORM_CNS_SCORE',
  'PRIMARY_INSTAL_AMT',
  'PRI_NO_OF_ACCTS',
  'UNIQUEID',
  'age',
  'disbursal_time'],
 'XG

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [15]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['AADHAR_FLAG', 'DRIVING_FLAG', 'PAN_FLAG', 'PASSPORT_FLAG', 'PRI_DISBURSED_AMOUNT', 'PRI_OVERDUE_ACCTS_bins', 'PRI_SANCTIONED_AMOUNT', 'UNIQUEID', 'VOTERID_FLAG', 'disbursal_time'], 'LDA': ['AADHAR_FLAG', 'DRIVING_FLAG', 'PAN_FLAG', 'PASSPORT_FLAG', 'PRI_DISBURSED_AMOUNT', 'PRI_OVERDUE_ACCTS_bins', 'PRI_SANCTIONED_AMOUNT', 'UNIQUEID', 'VOTERID_FLAG', 'disbursal_time'], 'DTC': ['ASSET_COST', 'ASSET_COST_new', 'CREDIT_HISTORY_LENGTH', 'DISBURSED_AMOUNT', 'DISBURSED_AMOUNT_new', 'LTV', 'PRI_NO_OF_ACCTS', 'UNIQUEID', 'age', 'disbursal_time'], 'RD': ['ASSET_COST', 'ASSET_COST_new', 'DISBURSED_AMOUNT', 'DISBURSED_AMOUNT_new', 'LTV', 'PERFORM_CNS_SCORE', 'PRI_NO_OF_ACCTS', 'UNIQUEID', 'age', 'disbursal_time'], 'LGBM': ['AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH', 'DISBURSED_AMOUNT', 'LTV', 'PERFORM_CNS_SCORE', 'PRIMARY_INSTAL_AMT', 'PRI_NO_OF_ACCTS', 'UNIQUEID', 'age', 'disbursal_time'], 'XGB': ['AVERAGE_ACCT_AGE', 'DRIVING_FLAG', 'PAN_FLAG', 'PASSPORT_FLAG', 'PRI_NO_OF_ACCTS', 